In [3]:
import os
import time
import copy
import torch
import torch.optim as optim
# 引入线性回归常用的损失函数
from torch.nn import Linear, Module, MSELoss
import pprint as pp
import utils.hypergraph_utils as hgut
from models import HGNN
from config import get_config
from datasets import load_feature_construct_H
from datasets import load_ft
import pandas as pd
import numpy as np
import math

In [9]:
t = np.ones((1072,1066)) * 1/1066

In [13]:
t1 = torch.tensor(t)
t2= torch.tensor(t)

l:\python3.6.6\lib\site-packages\ipykernel_launcher.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """Entry point for launching an IPython kernel.
l:\python3.6.6\lib\site-packages\ipykernel_launcher.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


In [20]:
t3 = torch.cat([t1,t2],dim= 1,out=None)

In [21]:
t3.shape

torch.Size([1072, 2132])

In [2]:
user_features = pd.read_csv(r"L:\hypergraph prediction\user_df.csv")

In [3]:
user_features = user_features.iloc[:,1:7]

In [4]:
# initialize data
data_dir = r"L:\HGNN try1\CG_dataset_rgrs.mat"
fts, lbls, idx_train, idx_test = load_ft(data_dir,'CGNN')

In [5]:
# H_dir
H_dir = r"L:\hypergraph prediction\话题用户超图.csv"
W_dir = r"L:\hypergraph prediction\用户超边权值.csv"
H = pd.read_csv(H_dir,header = None)
W = pd.read_csv(W_dir)
H = H.to_numpy()
W = W.iloc[:,1].to_numpy()

In [6]:
G = hgut.generate_G_from_H(H,W, variable_weight=False)

L:\HGNN try1\utils\hypergraph_utils.py:114: RuntimeWarning: divide by zero encountered in power
  DV2 = np.mat(np.diag(np.power(DV, -0.5)))


In [10]:
def generate_Gf_from_H(H,weight_matrix,edge_fts,variable_weight = False): # 从邻接矩阵构建超图
    """
    calculate Gf from hypgraph incidence matrix H
    :param H: hypergraph incidence matrix H
    :param variable_weight: whether the weight of hyperedge is variable
    :return: Gf: concat hyperedges features in first layer convolution
    """
    if type(H) != list:
        return _generate_Gf_from_H(H,weight_matrix,edge_fts,variable_weight)
    else:
        Gf = []
        for sub_H in H:
            Gf.append(generate_Gf_from_H(sub_H,weight_matrix,edge_fts,variable_weight))
        return Gf


def _generate_Gf_from_H(H, weight_matrix, edge_fts, variable_weight= False):
    """
    calculate G from hypgraph incidence matrix H
    :param H: hypergraph incidence matrix H
    :param variable_weight: whether the weight of hyperedge is variable
    :return: G
    """
    H = np.array(H)
    n_edge = H.shape[1]
    # the weight of the hyperedge
    W = weight_matrix
    W = (W - np.min(W)) / (np.max(W) - np.min(W))  # 节点权值归一化
    # the degree of the node
    DV = np.sum(H * W, axis=1)
    # the degree of the hyperedge
    DE = np.sum(H, axis=0)

    invDE = np.mat(np.diag(np.power(DE, -1)))
    DV2 = np.mat(np.diag(np.power(DV, -0.5)))
    W = np.mat(np.diag(W))
    H = np.mat(H)
    HT = H.T

    if variable_weight:
        DV2_H = DV2 * H
        invDE_HT_DV2 = invDE * HT * DV2
        return DV2_H, W, invDE_HT_DV2
    else:
        Y = W * invDE * HT * DV2
        edge_fts = edge_fts.values
        Yf = np.hstack([Y,edge_fts])
        Gf = DV2 * H * Yf
        return Gf


In [11]:
Gf = generate_Gf_from_H(H,W,user_features,
                           variable_weight=False )

l:\python3.6.6\lib\site-packages\ipykernel_launcher.py:35: RuntimeWarning: divide by zero encountered in power


[[0.00546082        nan 0.         ...        nan        nan 0.        ]
 [0.00533947        nan 0.         ...        nan        nan 0.        ]
 [0.00500575        nan 0.         ...        nan        nan 0.        ]
 ...
 [0.                nan 0.         ...        nan        nan 0.        ]
 [0.                nan 0.         ...        nan        nan 0.        ]
 [0.                nan 0.         ...        nan        nan 0.        ]] <class 'numpy.matrix'> <class 'pandas.core.frame.DataFrame'> (5022, 1066) (5022, 6)


In [12]:
Gf = torch.Tensor(Gf)
print(Gf)


tensor([[ 0.1177,     nan,  0.0000,  ..., 10.7403, 42.9613, 55.4917],
        [    nan,     nan,     nan,  ...,     nan,     nan,     nan],
        [ 0.0000,     nan,  0.2500,  ...,  9.9666, 29.8998, 19.9332],
        ...,
        [    nan,     nan,     nan,  ...,     nan,     nan,     nan],
        [    nan,     nan,     nan,  ...,     nan,     nan,     nan],
        [ 0.0000,     nan,  0.0000,  ...,  1.7009,  0.0000,  2.5514]])


In [13]:
Gf = torch.where(torch.isnan(Gf), torch.full_like(Gf, 0), Gf)
Gf

tensor([[ 0.1177,  0.0000,  0.0000,  ..., 10.7403, 42.9613, 55.4917],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.2500,  ...,  9.9666, 29.8998, 19.9332],
        ...,
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  1.7009,  0.0000,  2.5514]])

In [14]:
shape_trans = Linear(1072,1066)
Gf = shape_trans(Gf)

In [28]:
Gf.shape

torch.Size([1066, 1066])

In [16]:
G = torch.Tensor(G)

In [17]:
G = torch.where(torch.isnan(G), torch.full_like(G, 0), G)
G

tensor([[0.1177, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.2500,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0435]])

In [33]:
'''
fts, lbls, idx_train, idx_test, H = \
    load_feature_construct_H(data_dir,
                             m_prob=cfg['m_prob'],
                             K_neigs=cfg['K_neigs'],
                             is_probH=cfg['is_probH'],
                             use_mvcnn_feature=cfg['use_mvcnn_feature'],
                             use_gvcnn_feature=cfg['use_gvcnn_feature'],
                             use_mvcnn_feature_for_structure=cfg['use_mvcnn_feature_for_structure'],
                             use_gvcnn_feature_for_structure=cfg['use_gvcnn_feature_for_structure'])
'''

# n_class = int(lbls.max()) + 1
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu') # 指定存放的位置 cpu/gpu

# transform data to device
fts= torch.Tensor(fts).to(device)
lbls = torch.Tensor(lbls).squeeze().long().to(device)
G = torch.Tensor(G).to(device)

idx_train = torch.Tensor(idx_train).long().to(device)
idx_test = torch.Tensor(idx_test).long().to(device)

TypeError: expected TensorOptions(dtype=float, device=cpu, layout=Strided, requires_grad=false (default), pinned_memory=false (default), memory_format=(nullopt)) (got TensorOptions(dtype=__int64, device=cpu, layout=Strided, requires_grad=false (default), pinned_memory=false (default), memory_format=(nullopt)))

In [34]:
Gf = torch.Tensor(Gf).to(device)

In [38]:
def train_model(model, criterion,MAE, optimizer, scheduler, num_epochs=25, print_freq=500):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    # 分类任务的评估指标acc
#     best_acc = 0.0
    best_loss = 9999
    best_mrse = 9
    mae = 999
    mrse = 9999
    rmse = 999


    for epoch in range(num_epochs):
        # 间隔输出训练效果
        if epoch % print_freq == 0:
            print('-' * 10)
            print(f'Epoch {epoch}/{num_epochs - 1}')

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']: # 每一轮都训一遍再测一遍
            if phase == 'train':
                scheduler.step()
                model.train()  # Set model to training mode
            else:
                model.eval()  # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            idx = idx_train if phase == 'train' else idx_test # 选择相应数据集

            # Iterate over data.
            optimizer.zero_grad()
            with torch.set_grad_enabled(phase == 'train'):
                outputs = model(fts, G) # model 输出的是n*1
#                 print(outputs[idx].float())
                loss = criterion(outputs[idx].float(), lbls[idx].float()) # 根据自定义的criterion计算训练误差
                
                # _, preds = torch.max(outputs, 1)

                # backward + optimize only if in training phase 在训练阶段进行反向传播和优化
                if phase == 'train':
                    loss.backward()
                    optimizer.step()

            # statistics
            running_loss += loss.item() * fts.size(0)
#             print(fts.size(0))
            # running_corrects += torch.sum(preds[idx] == lbls.data[idx])

            epoch_loss = running_loss / len(idx)
#             epoch_acc = running_corrects.double() / len(idx)

            if epoch % print_freq == 0:
                print(f'{phase} Loss: {epoch_loss:.4f}')
            

            # deep copy the model 存储预测效果最好时候的model
            # deep copy 复制完整对象并且存储在独立的内存地址
            # 在评估阶段选择最好的模型，loss最小
            if phase == 'val' and epoch_loss < best_loss:
                best_loss = epoch_loss
#                 best_mrse = mrse
                best_model_wts = copy.deepcopy(model.state_dict())
   
                mae = MAE(outputs[idx].float(), lbls[idx].float())
                mrse = MRSE(outputs,lbls,idx).item()
                rmse = math.sqrt(best_loss)
#                 print(f'mrse: {mrse}')
#                 print(f'mae: {mae}')
#                 print(f'rmse: {rmse}')

        if epoch % print_freq == 0:
            print(f'Best val loss: {best_loss:4f}')
            print(f'mrse: {mrse}')
            print(f'mae: {mae}')
            print(f'rmse: {rmse}')
           
            print('-' * 20)

    time_elapsed = time.time() - since
    print(f'\nTraining complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val loss: {best_loss:4f}')
    print(f'mrse: {mrse}')
    print(f'mae: {mae}')
    print(f'rmse: {rmse}')

    # load best model weights
    model.load_state_dict(best_model_wts) # 读出最好的model并且返回
    return model

In [39]:
def _main():
    '''
    print(f"Classification on {cfg['on_dataset']} dataset!!! class number: {n_class}")
    print(f"use MVCNN feature: {cfg['use_mvcnn_feature']}")
    print(f"use GVCNN feature: {cfg['use_gvcnn_feature']}")
    print(f"use MVCNN feature for structure: {cfg['use_mvcnn_feature_for_structure']}")
    print(f"use GVCNN feature for structure: {cfg['use_gvcnn_feature_for_structure']}")
    print('Configuration -> Start')
    pp.pprint(cfg)
    print('Configuration -> End')
    '''

    # 定义模型，此处分类模型被耦合在HGNN函数中，需要去HGNN那里改预测任务
    model_ft = HGNN(in_ch=fts.shape[1], # 特征矩阵列数
                    n_class=1,
                    n_hid=128,
                    dropout=0)
    model_ft = model_ft.to(device) # 将模型转移到设备上

    # adam优化
    optimizer = optim.Adam(model_ft.parameters(), lr=0.01,weight_decay=0.005)
    # SGD优化
#     optimizer = optim.SGD(model_ft.parameters(), lr=0.01, weight_decay=cfg['weight_decay)'])

    # 动态调整学习率
    # milestone 调整轮次
    # gamma 调整倍数
    # new_lr = lr*gamma when epoch = milestone[i]
    schedular = optim.lr_scheduler.MultiStepLR(optimizer,
                                               milestones=[100,300,500,800],
                                               gamma=0.9)
    # criterion = torch.nn.CrossEntropyLoss() # 分类预测任务中使用了交叉熵损失函数
    # 线性回归要改成
    criterion = torch.nn.MSELoss(reduction = 'mean')
    MAE = torch.nn.L1Loss(reduction='mean')

    model_ft = train_model(model_ft, criterion,MAE, optimizer, schedular, 1500, print_freq=100)

In [40]:
def MRSE(outputs,lbls,idx):
    sums = 0
    count = len(outputs[idx])
    for i in range(count):
        truth = lbls[idx][i]
        pred = outputs[idx][i]
        delta =((truth-pred)/truth)*((truth-pred)/truth)
        sums = sums+delta
    res = sums / count
    return res

In [41]:
if __name__ == '__main__':
    _main()

----------
Epoch 0/1499


TypeError: forward() missing 1 required positional argument: 'Gf'

In [35]:
lbls

tensor([16,  8, 33,  ...,  8,  8, 18])

In [36]:
torch.mean(lbls.float())

tensor(20.4512)

In [37]:
torch.std(lbls.float())

tensor(46.9552)